In [2]:
%pip install liac-arff

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=53c06781e98038133a6fb642ae48f5d846a253fa89c087357ee47ae3241091f7
  Stored in directory: /Users/abdelkrimzitouni/Library/Caches/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff
Note: you may need to restart the kernel to use updated packages.


In [16]:
!gunzip -c ./ALOI/aloi-8d.csv.gz > ./ALOI/aloi-8d.csv
!gunzip -c ./ALOI/aloi-haralick-1.csv.gz > ./ALOI/aloi-haralick-1.csv
!gunzip -c ./ALOI/aloi-hsb-2x2x2.csv.gz > ./ALOI/aloi-hsb-2x2x2.csv

In [1]:
import os
import numpy as np
import pandas as pd
import arff

def read_aloi_directory(directory):
    file_list = os.listdir(directory)
    dataset_list = []
    last_column_values = []
    objs = arff.load(open(directory+'/objs.arff'))
    labels = np.array(objs['data'])
    sorted_labels = labels[np.argsort(labels[:, -1].astype(str))]
    sorted_labels[:, :-1] = sorted_labels[:, :-1].astype(float)
    last_column_label_values = sorted_labels[:, -1]
    for file in file_list:
        file_path = os.path.join(directory, file)
        
        if file.endswith('.csv'):
            df = pd.read_csv(file_path, delimiter=' ', header=None)
            df = df.dropna(axis=1, how='all')
            data = df.values
        elif file == "aloi-colorsim77.arff":
            dataset = arff.load(open(file_path))
            data = np.array(dataset['data'])
        else:
            continue
        
        sorted_dataset = data[np.argsort(data[:, -1].astype(str))]
        last_column_values = sorted_dataset[:, -1]
        
        if np.all(last_column_values == last_column_label_values):
            dataset_list.append(sorted_dataset)
    
    dataset_list.append(sorted_labels)
    return dataset_list


In [9]:
aloi = read_aloi_directory('./ALOI')

In [12]:
len(aloi)

5

In [15]:
os.makedirs('aloi_csv', exist_ok=True)
# Save each array as a CSV file
for i, arr in enumerate(aloi):
    df = pd.DataFrame(arr)
    # df = df.apply(pd.to_numeric, errors='ignore')
    arr = df.to_numpy()
    filename = f'aloi_csv/aloi_array_{i}.csv'
    print(i)
    np.savetxt(filename, arr, delimiter=',', fmt='%s')


0
1
2
3
4


In [29]:
mv_dataset_files = os.listdir("./aloi_csv")
mv_dataset_files.sort()
dataset = []
for file in mv_dataset_files:
    df = pd.read_csv(f"./aloi_csv/{file}", header=None)
    numerical_df = df.select_dtypes(include=[np.number])
    dataset.append(numerical_df.to_numpy())

dataset

[array([[8.86246293e-01, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 2.13464808e-02, 1.41262478e-02],
        [8.90708641e-01, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 1.85275608e-02, 1.68773510e-02],
        [8.92001682e-01, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 1.45331489e-02, 2.14549877e-02],
        ...,
        [9.85243056e-01, 4.36288339e-04, 3.25520833e-04, ...,
         0.00000000e+00, 1.23652705e-03, 1.18295175e-02],
        [9.85206887e-01, 5.53837529e-04, 4.06901042e-04, ...,
         3.61689815e-05, 1.47614656e-03, 1.12734194e-02],
        [9.85866970e-01, 6.12612124e-04, 6.87210648e-04, ...,
         2.48661748e-05, 1.38346354e-03, 1.03194625e-02]]),
 array([[0.52106334, 0.5549817 , 0.55797701, ..., 0.65483649, 0.59232047,
         0.51372448],
        [0.51680537, 0.55127803, 0.55560497, ..., 0.65723221, 0.59390096,
         0.51455813],
        [0.51327497, 0.54751493, 0.55261007, ..., 0.6617243 , 0.59765731,